In [1]:
import pandas as pd
from statistics import mean, median
import sys
import pprint
import pickle

In [2]:
def output_values(Y_data):
    Y_t = []
    for e in Y_data:
        if e == 'passed':
            Y_t.append(1)
        else:
            Y_t.append(0) 
    return Y_t

In [3]:
def get_first_failures(df):
    
    results = df['tr_status'].tolist()
    length = len(results)
    verdict = ['keep']
    prev = results[0]
    
    for i in range(1, length):
        if results[i] == 0:
            if prev == 0:
                verdict.append('discard')
                #print(i+1)
            else:
                verdict.append('keep')
        else:
            verdict.append('keep')
        prev = results[i]
    
    df['verdict'] = verdict
    df = df[ df['verdict'] == 'keep' ]
    df.drop('verdict', inplace=True, axis=1)
    return df

In [4]:
def str_to_list(s):
    if s == '[]':
        return [0]
    l = s[1:-1].split(', ')
    l = [int(x) for x in l]
    return l

In [5]:
def separate_versions(results):
    version_dfs = []
    
    for i in range(1,11):
        ver = results[ results['version']==i]
        version_dfs.append(ver)
    return version_dfs

In [26]:
def start_result_collection(filename):
    results = pd.read_csv(filename)
    
    projects = set(results['project'].tolist())
    all_versions = pd.DataFrame()
        
    for p in projects:
        
        p_data = results[ results['project']==p]
        pframe = pd.DataFrame()
        
        versions = separate_versions(p_data)
        
        for start in range(0,10):
            if len(versions[start]) > 0:
                pframe = versions[start]
                break
        
        for x in range(len(pframe)):
            row = pframe.iloc[x]
            
            alg = row['algorithm']
            b = row['batch_size']
            conf = row['confidence']
            
            final_proj_delays = str_to_list(row['project_delays'])
            final_batch_median = str_to_list(row['batch_median'])
            final_ci = str_to_list(row['ci'])
            
            
            for i in range(start+1,10):
                next_ver = versions[i]
                new_df = next_ver[ (next_ver['algorithm']==alg) & (next_ver['batch_size']==b) & (next_ver['confidence']==conf)]
                
                if len(new_df) > 0:
                    new_row = new_df.iloc[0]
                else:
                    continue
                
                total_reqd_builds = (row['project_reqd_builds']*row['testall_size']) + (new_row['project_reqd_builds']*new_row['testall_size'])
                total_missed_builds = (row['project_missed_builds']*row['testall_size']) + (new_row['project_missed_builds']*new_row['testall_size'])
                total_saved_builds = (row['project_saved_builds']*row['testall_size']) + (new_row['project_saved_builds']*new_row['testall_size'])
                total_size = row['testall_size'] + new_row['testall_size']
                
                row['project_reqd_builds'] = total_reqd_builds/total_size
                row['project_missed_builds'] = total_missed_builds/total_size
                row['project_saved_builds'] = total_saved_builds/total_size
                row['testall_size'] = total_size
                
#                 print(len(final_ci))
#                 print(len(str_to_list(new_row['ci'])))
#                 print('----')
                
                final_proj_delays.extend(str_to_list(new_row['project_delays']))
                final_batch_median.extend(str_to_list(new_row['batch_median']))
                final_ci.extend(str_to_list(new_row['ci']))
                
#                 print(len(final_ci))
#                 print('====')
                row['project_delays'] = final_proj_delays
                row['batch_median'] = final_batch_median
                row['ci'] = final_ci
                
                
            pframe.iloc[x] = row
        all_versions = all_versions.append(pframe)
    
    return all_versions

In [27]:
filenames = ['cvh_models.csv', 'lfs.csv', 'results1.csv', 'results2.csv', 'gradle_models.csv', 'rubinius_models.csv']
file_root = '../version_results/'
dfs = []

for file in filenames:
    dfs.append(start_result_collection(file_root+file))

#combining dfs
final_df = pd.DataFrame()
final_df = dfs[0]

for i in range(1, len(dfs)):
    final_df = final_df.append(dfs[i])

final_df.to_csv('final_results.csv')

In [34]:
final_df = final_df.loc[:, ~final_df.columns.str.contains('^Unnamed')]

In [37]:
final_df.to_csv('final_results.csv')

In [38]:
short = pd.DataFrame()
short = final_df

In [41]:
short.drop('ci', inplace=True, axis=1)

/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [43]:
final_df.to_csv('int_results.csv')